In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [4]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [5]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../saved_models/{model_name}'))

In [6]:
gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U0A0_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_1 = gen(to_gen).detach().numpy()
y_1 = np.zeros(35)

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U0A1_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_2 = gen(to_gen).detach().numpy()
y_2 = np.ones(35)

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U0A2_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_3 = gen(to_gen).detach().numpy()
y_3 = np.ones(35) + 1

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U1A0_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_4 = gen(to_gen).detach().numpy()
y_4 = np.ones(35) + 2

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U1A1_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_5 = gen(to_gen).detach().numpy()
y_5 = np.ones(35) + 3

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U1A2_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_6 = gen(to_gen).detach().numpy()
y_6 = np.ones(35) + 4

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U2A0_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_7 = gen(to_gen).detach().numpy()
y_7 = np.ones(35) + 5

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U2A1_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_8 = gen(to_gen).detach().numpy()
y_8 = np.ones(35) + 6

gen = Generator(z_dim = 106)
gen.eval()
load_model(gen, "U2A2_gen.param")
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_9 = gen(to_gen).detach().numpy()
y_9 = np.ones(35) + 7

X_train = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
y_train = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

In [7]:
activities = [1, 3, 4]
users = [1, 3, 5]
X_test, y_test = start_data(activities, users, "Activity", sub_features, act_features)
_, X_test, _, y_test = train_test_split(X_test, y_test, test_size = 0.2, shuffle = True)

In [8]:
model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.413949251174927, Final Batch Loss: 2.2008166313171387
Epoch 2, Loss: 4.431711673736572, Final Batch Loss: 2.2269327640533447
Epoch 3, Loss: 4.416304588317871, Final Batch Loss: 2.2068591117858887
Epoch 4, Loss: 4.40888524055481, Final Batch Loss: 2.199678659439087
Epoch 5, Loss: 4.407526969909668, Final Batch Loss: 2.198256492614746
Epoch 6, Loss: 4.407911539077759, Final Batch Loss: 2.199852228164673
Epoch 7, Loss: 4.419575214385986, Final Batch Loss: 2.2201623916625977
Epoch 8, Loss: 4.387150287628174, Final Batch Loss: 2.1789753437042236
Epoch 9, Loss: 4.408385276794434, Final Batch Loss: 2.211353302001953
Epoch 10, Loss: 4.398369789123535, Final Batch Loss: 2.2031071186065674
Epoch 11, Loss: 4.397565841674805, Final Batch Loss: 2.200876235961914
Epoch 12, Loss: 4.394002676010132, Final Batch Loss: 2.2080044746398926
Epoch 13, Loss: 4.383869171142578, Final Batch Loss: 2.2001445293426514
Epoch 14, Loss: 4.354028701782227, Final Batch Loss: 2.1690430641174316
Epoch 1

Epoch 133, Loss: 2.148605227470398, Final Batch Loss: 1.0656206607818604
Epoch 134, Loss: 2.2219574451446533, Final Batch Loss: 1.1627492904663086
Epoch 135, Loss: 2.064661741256714, Final Batch Loss: 1.0076119899749756
Epoch 136, Loss: 2.1647006273269653, Final Batch Loss: 1.1457512378692627
Epoch 137, Loss: 2.120636463165283, Final Batch Loss: 1.0430405139923096
Epoch 138, Loss: 2.1041252613067627, Final Batch Loss: 1.0282894372940063
Epoch 139, Loss: 2.1791491508483887, Final Batch Loss: 1.1174991130828857
Epoch 140, Loss: 2.077961564064026, Final Batch Loss: 1.0573025941848755
Epoch 141, Loss: 2.0254147052764893, Final Batch Loss: 1.0610508918762207
Epoch 142, Loss: 1.9701151251792908, Final Batch Loss: 0.9610900282859802
Epoch 143, Loss: 1.970308005809784, Final Batch Loss: 1.044114112854004
Epoch 144, Loss: 2.176750063896179, Final Batch Loss: 1.1239242553710938
Epoch 145, Loss: 1.9815624356269836, Final Batch Loss: 0.937921941280365
Epoch 146, Loss: 1.9613284468650818, Final Bat

Epoch 262, Loss: 1.1759455800056458, Final Batch Loss: 0.5237343907356262
Epoch 263, Loss: 1.2374668717384338, Final Batch Loss: 0.5326216816902161
Epoch 264, Loss: 1.318650245666504, Final Batch Loss: 0.7443643808364868
Epoch 265, Loss: 1.2007532119750977, Final Batch Loss: 0.6236628293991089
Epoch 266, Loss: 1.210332989692688, Final Batch Loss: 0.5408207178115845
Epoch 267, Loss: 1.3074212670326233, Final Batch Loss: 0.6441075205802917
Epoch 268, Loss: 1.3648419976234436, Final Batch Loss: 0.7036536335945129
Epoch 269, Loss: 1.3689398765563965, Final Batch Loss: 0.6968533992767334
Epoch 270, Loss: 1.2337549328804016, Final Batch Loss: 0.6071485280990601
Epoch 271, Loss: 1.2706822156906128, Final Batch Loss: 0.6235003471374512
Epoch 272, Loss: 1.2381436228752136, Final Batch Loss: 0.5838121771812439
Epoch 273, Loss: 1.2226536273956299, Final Batch Loss: 0.5680387616157532
Epoch 274, Loss: 1.2038246393203735, Final Batch Loss: 0.5902066826820374
Epoch 275, Loss: 1.178099513053894, Fina

Epoch 391, Loss: 0.5839110314846039, Final Batch Loss: 0.2530598044395447
Epoch 392, Loss: 0.5263825356960297, Final Batch Loss: 0.23349860310554504
Epoch 393, Loss: 0.597059965133667, Final Batch Loss: 0.333942711353302
Epoch 394, Loss: 0.4945167452096939, Final Batch Loss: 0.2485407292842865
Epoch 395, Loss: 0.4382820278406143, Final Batch Loss: 0.20833449065685272
Epoch 396, Loss: 0.48337221145629883, Final Batch Loss: 0.2543613314628601
Epoch 397, Loss: 0.49173930287361145, Final Batch Loss: 0.19750818610191345
Epoch 398, Loss: 0.5439663231372833, Final Batch Loss: 0.28462886810302734
Epoch 399, Loss: 0.49111609160900116, Final Batch Loss: 0.2610728442668915
Epoch 400, Loss: 0.5370165705680847, Final Batch Loss: 0.2609894275665283
Epoch 401, Loss: 0.5133425295352936, Final Batch Loss: 0.22650936245918274
Epoch 402, Loss: 0.45670609176158905, Final Batch Loss: 0.18724869191646576
Epoch 403, Loss: 0.5441243797540665, Final Batch Loss: 0.31270259618759155
Epoch 404, Loss: 0.6136711239

Epoch 521, Loss: 0.313468798995018, Final Batch Loss: 0.14162860810756683
Epoch 522, Loss: 0.3610745221376419, Final Batch Loss: 0.18508115410804749
Epoch 523, Loss: 0.3074585646390915, Final Batch Loss: 0.1536046266555786
Epoch 524, Loss: 0.3905794620513916, Final Batch Loss: 0.2013835906982422
Epoch 525, Loss: 0.24903105944395065, Final Batch Loss: 0.10178273171186447
Epoch 526, Loss: 0.22354263812303543, Final Batch Loss: 0.10038977861404419
Epoch 527, Loss: 0.32604360580444336, Final Batch Loss: 0.14108778536319733
Epoch 528, Loss: 0.28754329681396484, Final Batch Loss: 0.11028532683849335
Epoch 529, Loss: 0.44519568979740143, Final Batch Loss: 0.3073287606239319
Epoch 530, Loss: 0.42404040694236755, Final Batch Loss: 0.21934862434864044
Epoch 531, Loss: 0.27606096118688583, Final Batch Loss: 0.11494158953428268
Epoch 532, Loss: 0.3825228214263916, Final Batch Loss: 0.2205248773097992
Epoch 533, Loss: 0.41063156723976135, Final Batch Loss: 0.2447146624326706
Epoch 534, Loss: 0.2198

...

Epoch 4440, Loss: 0.015995396301150322, Final Batch Loss: 0.008751487359404564
Epoch 4441, Loss: 0.008970635710284114, Final Batch Loss: 0.002385397208854556
Epoch 4442, Loss: 0.0312784796115011, Final Batch Loss: 0.002329412614926696
Epoch 4443, Loss: 0.014665356371551752, Final Batch Loss: 0.004816846456378698
Epoch 4444, Loss: 0.018713680561631918, Final Batch Loss: 0.01133857574313879
Epoch 4445, Loss: 0.01822551153600216, Final Batch Loss: 0.007228285074234009
Epoch 4446, Loss: 0.00989909190684557, Final Batch Loss: 0.0035260245203971863
Epoch 4447, Loss: 0.013161727460101247, Final Batch Loss: 0.0026576111558824778
Epoch 4448, Loss: 0.00905604287981987, Final Batch Loss: 0.005189787596464157
Epoch 4449, Loss: 0.046328090131282806, Final Batch Loss: 0.03421146050095558
Epoch 4450, Loss: 0.014547926839441061, Final Batch Loss: 0.007305283099412918
Epoch 4451, Loss: 0.047048079781234264, Final Batch Loss: 0.03243772312998772
Epoch 4452, Loss: 0.03488900698721409, Final Batch Loss: 0

Epoch 4551, Loss: 0.05547057744115591, Final Batch Loss: 0.04284088313579559
Epoch 4552, Loss: 0.013406615238636732, Final Batch Loss: 0.008154496550559998
Epoch 4553, Loss: 0.023034241050481796, Final Batch Loss: 0.010441129095852375
Epoch 4554, Loss: 0.02098484057933092, Final Batch Loss: 0.005651518702507019
Epoch 4555, Loss: 0.03725062683224678, Final Batch Loss: 0.028102343901991844
Epoch 4556, Loss: 0.02292789425700903, Final Batch Loss: 0.011816931888461113
Epoch 4557, Loss: 0.026384311728179455, Final Batch Loss: 0.02095942199230194
Epoch 4558, Loss: 0.014427911955863237, Final Batch Loss: 0.005429826211184263
Epoch 4559, Loss: 0.033824807964265347, Final Batch Loss: 0.025425953790545464
Epoch 4560, Loss: 0.01225450192578137, Final Batch Loss: 0.0036748137790709734
Epoch 4561, Loss: 0.02402372471988201, Final Batch Loss: 0.008810858242213726
Epoch 4562, Loss: 0.005960999988019466, Final Batch Loss: 0.002226489130407572
Epoch 4563, Loss: 0.009903803002089262, Final Batch Loss: 0

Epoch 4676, Loss: 0.02170246746391058, Final Batch Loss: 0.012469543144106865
Epoch 4677, Loss: 0.024748736061155796, Final Batch Loss: 0.0030968161299824715
Epoch 4678, Loss: 0.0343352637719363, Final Batch Loss: 0.0022960014175623655
Epoch 4679, Loss: 0.02745119296014309, Final Batch Loss: 0.005754653364419937
Epoch 4680, Loss: 0.05284066451713443, Final Batch Loss: 0.04764990136027336
Epoch 4681, Loss: 0.04447172209620476, Final Batch Loss: 0.01693810150027275
Epoch 4682, Loss: 0.011436871718615294, Final Batch Loss: 0.006268599536269903
Epoch 4683, Loss: 0.014367723953910172, Final Batch Loss: 0.001671253819949925
Epoch 4684, Loss: 0.022215277887880802, Final Batch Loss: 0.012517857365310192
Epoch 4685, Loss: 0.028956088237464428, Final Batch Loss: 0.016192171722650528
Epoch 4686, Loss: 0.007734199520200491, Final Batch Loss: 0.004439669661223888
Epoch 4687, Loss: 0.018504141829907894, Final Batch Loss: 0.0058763958513736725
Epoch 4688, Loss: 0.024804630782455206, Final Batch Loss:

Epoch 4787, Loss: 0.007622437318786979, Final Batch Loss: 0.004032039549201727
Epoch 4788, Loss: 0.015633638482540846, Final Batch Loss: 0.0065560718066990376
Epoch 4789, Loss: 0.0298927859403193, Final Batch Loss: 0.02450314350426197
Epoch 4790, Loss: 0.010138883721083403, Final Batch Loss: 0.00394267775118351
Epoch 4791, Loss: 0.019383170874789357, Final Batch Loss: 0.0021095287520438433
Epoch 4792, Loss: 0.03018774464726448, Final Batch Loss: 0.021113183349370956
Epoch 4793, Loss: 0.020962849259376526, Final Batch Loss: 0.011864950880408287
Epoch 4794, Loss: 0.034352818969637156, Final Batch Loss: 0.005605789367109537
Epoch 4795, Loss: 0.009084594901651144, Final Batch Loss: 0.004857964348047972
Epoch 4796, Loss: 0.011196876410394907, Final Batch Loss: 0.00242116441950202
Epoch 4797, Loss: 0.07223477913066745, Final Batch Loss: 0.06576424092054367
Epoch 4798, Loss: 0.02601418551057577, Final Batch Loss: 0.01942754164338112
Epoch 4799, Loss: 0.012088938150554895, Final Batch Loss: 0.

Epoch 4900, Loss: 0.011063859332352877, Final Batch Loss: 0.006123675033450127
Epoch 4901, Loss: 0.03161316458135843, Final Batch Loss: 0.010086451657116413
Epoch 4902, Loss: 0.033180566504597664, Final Batch Loss: 0.017816094681620598
Epoch 4903, Loss: 0.007738381274975836, Final Batch Loss: 0.0013218960957601666
Epoch 4904, Loss: 0.024531796108931303, Final Batch Loss: 0.0022759917192161083
Epoch 4905, Loss: 0.018468664959073067, Final Batch Loss: 0.003971543163061142
Epoch 4906, Loss: 0.025271897204220295, Final Batch Loss: 0.009757359512150288
Epoch 4907, Loss: 0.013554560951888561, Final Batch Loss: 0.0021198326721787453
Epoch 4908, Loss: 0.023295882623642683, Final Batch Loss: 0.016993563622236252
Epoch 4909, Loss: 0.01353152608498931, Final Batch Loss: 0.009072506800293922
Epoch 4910, Loss: 0.014622297137975693, Final Batch Loss: 0.0030295420438051224
Epoch 4911, Loss: 0.006942389765754342, Final Batch Loss: 0.0018031119834631681
Epoch 4912, Loss: 0.010925285052508116, Final Bat

In [54]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 3))

[[12  0  1  3  0  0  0  0  0]
 [ 0  6  0  0  3  0  0  2  0]
 [ 0  0  5  0  0  3  0  0  1]
 [ 0  0  0  9  0  0  0  0  0]
 [ 1  2  0  0  6  0  0  1  0]
 [ 0  0  2  0  0  5  0  0  4]
 [ 0  0  0  0  0  0 12  3  0]
 [ 0  1  0  0  0  0  0  8  0]
 [ 0  0  5  0  0  2  0  0  3]]
              precision    recall  f1-score   support

           0      0.923     0.750     0.828        16
           1      0.667     0.545     0.600        11
           2      0.385     0.556     0.455         9
           3      0.750     1.000     0.857         9
           4      0.667     0.600     0.632        10
           5      0.500     0.455     0.476        11
           6      1.000     0.800     0.889        15
           7      0.571     0.889     0.696         9
           8      0.375     0.300     0.333        10

    accuracy                          0.660       100
   macro avg      0.649     0.655     0.641       100
weighted avg      0.684     0.660     0.661       100



In [55]:
16+11+9+10+15+9

70

In [56]:
12+6+9+6+12+8

53

In [57]:
53/70

0.7571428571428571